# 问答

## 用例
此处展示了如何使用 Langchian + 千帆 SDK 完成对特定文档完成获取、切分、转为向量并存储，而后根据你的提问来从文中获取答案。
并且借助 Langsmith 将整个过程可视化展现

## 概览
把一个非结构化的文档转成问答链涉及以下步骤：
1. `Loading`: 首先我们需要加载数据，非结构化的数据可以从多种渠道加载。点击 [LangChain integration hub](https://integrations.langchain.com/) 查看所有 Langchain 支持的 Loader。
每个 Loader 都会返回 Langchian 中的 [`Document`](/docs/components/schema/document) 对象。

2. `Splitting`: [文本切分器](/docs/modules/data_connection/document_transformers/) 把 `Documents` 切分成特定的大小。

3. `Storage`: `Storage` （例如 [vectorstore](/docs/modules/data_connection/vectorstores/)）会将切分的数据储存起来，通常还附带对文本做 [embedding](https://www.pinecone.io/learn/vector-embeddings/) 。

4. `Retrieval`: 用于从 `Storage` 中获取切分的数据，用于后面生成答案。

5. `Generation`: 使用提示词和获取到的数据，搭配 [LLM](/docs/modules/model_io/models/llms/) 来生成回答。

6. `Conversation` (扩展): 添加 [Memory](/docs/modules/memory/) 模块来在你的问答链上实现多轮对话。

![flow.jpeg](img/qa_flow.jpeg)

接下来我们会演示如何一步步构造我们自己的流水线，并且实现我们自己定制化的功能

## Step 0. Prepare

为了能够运行我们的 Demo，首先我们需要下载以下对应版本的依赖并且设置环境变量

In [1]:
!pip install -U langchain
!pip install -U chromadb
!pip install -U qianfan
!pip install -U pdfplumber

  Using cached chroma_hnswlib-0.7.3-cp310-cp310-macosx_11_0_arm64.whl.metadata (252 bytes)
  Using cached PyPika-0.48.9-py2.py3-none-any.whl
  Using cached asgiref-3.8.1-py3-none-any.whl.metadata (9.3 kB)
  Using cached httptools-0.6.1-cp310-cp310-macosx_10_9_universal2.whl.metadata (3.6 kB)
  Using cached uvloop-0.19.0-cp310-cp310-macosx_10_9_universal2.whl.metadata (4.9 kB)
  Using cached watchfiles-0.21.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (4.9 kB)
  Using cached websockets-12.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 2.3 MB/s eta 0:00:00a 0:00:01m
Using cached chroma_hnswlib-0.7.3-cp310-cp310-macosx_11_0_arm64.whl (197 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.5/528.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 12.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip list | grep -E 'qianfan|langchain|chromadb|pdfplumber'

chromadb                                 0.4.24
langchain                                0.1.13
langchain-community                      0.0.29
langchain-core                           0.1.35
langchain-text-splitters                 0.0.1
pdfplumber                               0.11.0
qianfan                                  0.3.6.1           /Users/pengyiyang/Desktop/github/bce-qianfan-sdk/python


In [3]:
import os

os.environ['QIANFAN_AK'] = "your_ak"
os.environ['QIANFAN_SK'] = "your_sk"

# 此处为 Langsmith 相关功能开关。当且仅当你知道这是做什么用时，可删除注释并设置变量以使用 Langsmith 相关功能
# os.environ['LANGCHAIN_TRACING_V2'] = "true"
# os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
# os.environ['LANGCHAIN_API_KEY'] = "LANGCHAIN_API_KEY"
# os.environ['LANGCHAIN_PROJECT'] = "LANGCHAIN_PROJECT"


is_chinese = True

if is_chinese:
    WEB_URL = "https://zhuanlan.zhihu.com/p/85289282"
    CUSTOM_PROMPT_TEMPLATE = """
        使用下面的语料来回答本模板最末尾的问题。如果你不知道问题的答案，直接回答 "我不知道"，禁止随意编造答案。
        为了保证答案尽可能简洁，你的回答必须不超过三句话。
        请注意！在每次回答结束之后，你都必须接上 "感谢你的提问" 作为结束语
        以下是一对问题和答案的样例：
            请问：秦始皇的原名是什么
            秦始皇原名嬴政。感谢你的提问。
        
        以下是语料：
        
        {context}
        
        请问：{question}
    """
    QUESTION1 = "明朝的开国皇帝是谁"
    QUESTION2 = "朱元璋是什么时候建立的明朝"
else:
    WEB_URL = "https://lilianweng.github.io/posts/2023-06-23-agent/"
    CUSTOM_PROMPT_TEMPLATE = """
        Use the following pieces of context to answer the question at the end. 
        If you don't know the answer, just say that you don't know, don't try to make up an answer. 
        Use three sentences maximum and keep the answer as concise as possible. 
        Always say "thanks for asking!" at the end of the answer. 
        {context}
        Question: {question}
        Helpful Answer:
    """
    QUESTION1 = "How do agents use Task decomposition?"
    QUESTION2 = "What are the various ways to implemet memory to support it?"

## Step 1. Load

指定一个 `DocumentLoader` 来把你指定的非结构化数据加载成 `Documents`。一个 `Document` 是文字（即 `page_content`）和与之相关的元数据的结合体

此处我们使用 `WebBaseLoader` ，从网页中加载一个 `Documents`。

In [4]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader(WEB_URL)
data = loader.load()

Langchain 内提供了非常多样的 Loader ，辅助用户从不同来源读取数据。这些 Loader 都声明在 `langchain.document_loaders` 包中。

对于我们的中文示例，我们还提供了一种从 PDF 读取 `Document` 的演示样例：

In [5]:
# ! pip install pdfplumber
from langchain.document_loaders import PDFPlumberLoader

if is_chinese:
    loader = PDFPlumberLoader("example_data/中国古代史-明朝.pdf")
    data = loader.load()

## Step 2. Split

接下来把 `Document` 切分成块，为后续的 embedding 和存入向量数据库做准备。

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap = 0, separators=["\n\n", "\n", " ", "", "。", "，"])
all_splits = text_splitter.split_documents(data)

## Step 3. Store

为了能够查询文档的片段，我们首先需要把它们存储起来，一种比较常见的做法是对文档的内容做 embedding，然后再将 embedding 的向量连同文档一起存入向量数据库中，此处 embedding 用于索引文档。

In [7]:
from langchain.embeddings import QianfanEmbeddingsEndpoint
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=all_splits, embedding=QianfanEmbeddingsEndpoint())

[INFO] [03-28 18:24:29] openapi_requestor.py:336 [t:8094817088]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-28 18:24:30] openapi_requestor.py:336 [t:8094817088]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-28 18:24:31] openapi_requestor.py:336 [t:8094817088]: requesting llm api endpoint: /embeddings/embedding-v1


除了非结构化的文档以外，Langchain 还可以从多种数据源获取数据并将它们存储起来。

![lc.png](img/qa_data_load.png)

## Step 4. Retrieve

我们可以使用 [相似度搜索](https://www.pinecone.io/learn/what-is-similarity-search/) 来从切分的文档内获取数据，获取到的数据会作为最终提交给 LLM 的 prompt 的一部分。

In [8]:
docs = vectorstore.similarity_search_with_relevance_scores(QUESTION1)
[(document.page_content, score) for document, score in docs]

[INFO] [03-28 18:24:36] openapi_requestor.py:336 [t:8094817088]: requesting llm api endpoint: /embeddings/embedding-v1


[('中国古代史-明朝（1368~1644）\n历史⻛云\n关注他\n看历史，就看历史⻛云\n起源\n元朝末年蒙古统治者残暴，⼈⺠受到空前压迫。1351年，元廷征调农⺠和兵⼠⼗⼏万⼈治理⻩河⽔\n患。“治河”和“变钞”导致红⼱军起义爆发。\n郭⼦兴\n1351年5⽉，韩⼭童、刘福通领导红⼱军起义爆发。次年，郭⼦兴响应，聚众起义，攻占濠州。平\n⺠出身的朱元璋受汤和邀请投奔郭⼦兴，屡⽴战功，备受郭⼦兴器重和信任，并娶郭⼦兴养⼥⻢⽒\n为妻。不久，朱元璋离开濠州，发展⾃⼰的势⼒。1355年，刘福通⽴韩林⼉为帝，国号宋，年号\n⻰凤，称⼩明王，以亳州为都城。郭⼦兴病故后朱元璋统率郭部，⼩明王任其为左副元帅。',
  0.39380364803544865),
 ('李⾃成建国⼤顺，三⽉，李⾃成率军北伐攻陷⼤同、宣府、居庸关，最后攻克北京。崇祯在煤⼭⾃\n缢，明朝作为统⼀国家的历史结束。\n南明时期\n北京沦陷后，史可法等⼈在南京拥⽴福王朱由崧，建⽴弘光政权，即弘光帝，弘光帝死后，鲁王朱\n以海于浙江绍兴监国；⽽唐王朱⾀键在郑芝⻰等⼈的拥⽴下，于福建福州称帝，即隆武帝。⽽两个\n南明政权为争夺正统地位互相攻伐。1651年在⾈⼭群岛沦陷后，鲁王朱以海在张名振、张煌⾔陪同\n下，赴厦⻔依靠郑成功，不久病死在⾦⻔。隆武帝屡议出师北伐，因得不到郑芝⻰的⽀持⽽⽆疾⽽\n终。1646年，清军攻占浙江与福建，鲁王逃亡海外，隆武帝于汀州逃往江⻄时被俘，绝⻝殉国，',
  0.33035692025205854),
 ('戚继光\n嘉靖⼀朝，国家外患不断。北⽅鞑靼趁明朝衰弱⽽⼊据河套。1550年鞑靼⾸领俺答进犯⼤同，宣\n⼤总兵仇鸾重⾦收买俺答，让其转向其他⽬标。结果俺答转⽽直攻北京，在北京城郊⼤肆抢掠后⻄\n去，明朝军队在追击过程中战败，此为庚戌之变。东南沿海由⽇本浪⼈与中国海盗组成的倭寇与沿\n海居⺠合作⾛私，先并且后袭扰⼭东、浙江、福建与⼴东等地区。戚继光与俞⼤猷平定浙闽粤等地\n的倭寇，为隆庆开关奠定基础。另葡萄⽛⼈于1557年移⺠澳⻔，但及⾄明亡，葡萄⽛⼈及澳⻔始\n终为⼴东布政司⾹⼭县管辖。1566年明世宗驾崩，皇太⼦朱载垕即位，即明穆宗，年号隆庆，翌\n年为隆庆元年。',
  0.31617399996596063),
 ('明惠宗\n1398年朱元璋驾崩，由于太⼦朱标早死，由

## Step 5. Generate

接下来我们就可以使用我们的大模型（例如文心一言）和 Langchain 的 RetrievalQA 链，来针对这篇文档进行提问并获取我们想要的回答了。

In [10]:
from langchain.chains import RetrievalQA
from langchain.chat_models import QianfanChatEndpoint
from langchain.prompts import PromptTemplate

QA_CHAIN_PROMPT = PromptTemplate.from_template(CUSTOM_PROMPT_TEMPLATE)

llm = QianfanChatEndpoint()
retriever=vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={'score_threshold': 0.0})
                                   
qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever, chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})
qa_chain.invoke({"query": QUESTION1})

[INFO] [03-28 18:24:59] openapi_requestor.py:336 [t:8094817088]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-28 18:24:59] openapi_requestor.py:336 [t:8094817088]: requesting llm api endpoint: /chat/eb-instant


{'query': '明朝的开国皇帝是谁', 'result': '明朝的开国皇帝是朱元璋。感谢你的提问。'}

在上面的执行中，我们使用了千帆平台上提供的大模型调用，成功就一个问进行了问答。

此外，`RetrievalQA` 链中使用的 `prompt` 参数也是可以定制的。由于 Langchain `RetrievalQA` 链中默认提供的 prompt 是用英语编写的，所以此处我们替换为了我们手动实现的中文 prompt，针对中文语境进行优化。

#### 使用不同的大模型


除了使用默认的模型，即 `ERNIE-Bot-turbo` 以外，用户还可以设置上面 `QianfanChatEndpoint` 的 `model` 参数，来指定使用不同的大模型。例如我们想使用 ERNIE-Bot-4 模型时，就可以这么设置：

In [11]:
llm = QianfanChatEndpoint(model="ERNIE-Bot-4")

qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever, chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})
qa_chain.invoke({"query": QUESTION1})

[INFO] [03-28 18:25:06] openapi_requestor.py:336 [t:8094817088]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-28 18:25:06] openapi_requestor.py:336 [t:8094817088]: requesting llm api endpoint: /chat/completions_pro


{'query': '明朝的开国皇帝是谁', 'result': '明朝的开国皇帝是朱元璋。感谢你的提问。'}

或者如果你已经在千帆平台上购买了资源并部署了自己的大模型服务，千帆 Langchain 组件还提供了 `endpoint` 参数，让你能够在 Langchian 中调用自己微调的大模型。有条件的用户可以取消注释并修改下列代码进行体验。

In [ ]:
# llm = QianfanChatEndpoint(endpoint="your_service_endpoint")

# qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever, chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})
# qa_chain.invoke({"query": QUESTION1})

#### 返回源文档

用于 QA 的知识文档也可以通过指定 `return_source_documents=True` 被包含在返回的字典里

In [12]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever, chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}, return_source_documents=True)
result = qa_chain.invoke({"query": QUESTION1})
len(result['source_documents'])
result['source_documents']

[INFO] [03-28 18:25:33] openapi_requestor.py:336 [t:8094817088]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-28 18:25:34] openapi_requestor.py:336 [t:8094817088]: requesting llm api endpoint: /chat/completions_pro


[Document(page_content='中国古代史-明朝（1368~1644）\n历史⻛云\n关注他\n看历史，就看历史⻛云\n起源\n元朝末年蒙古统治者残暴，⼈⺠受到空前压迫。1351年，元廷征调农⺠和兵⼠⼗⼏万⼈治理⻩河⽔\n患。“治河”和“变钞”导致红⼱军起义爆发。\n郭⼦兴\n1351年5⽉，韩⼭童、刘福通领导红⼱军起义爆发。次年，郭⼦兴响应，聚众起义，攻占濠州。平\n⺠出身的朱元璋受汤和邀请投奔郭⼦兴，屡⽴战功，备受郭⼦兴器重和信任，并娶郭⼦兴养⼥⻢⽒\n为妻。不久，朱元璋离开濠州，发展⾃⼰的势⼒。1355年，刘福通⽴韩林⼉为帝，国号宋，年号\n⻰凤，称⼩明王，以亳州为都城。郭⼦兴病故后朱元璋统率郭部，⼩明王任其为左副元帅。', metadata={'CreationDate': "D:20230922071555+00'00'", 'Creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36', 'ModDate': "D:20230922071555+00'00'", 'Producer': 'Skia/PDF m114', 'file_path': 'example_data/中国古代史-明朝.pdf', 'page': 0, 'source': 'example_data/中国古代史-明朝.pdf', 'total_pages': 16}),
 Document(page_content='李⾃成建国⼤顺，三⽉，李⾃成率军北伐攻陷⼤同、宣府、居庸关，最后攻克北京。崇祯在煤⼭⾃\n缢，明朝作为统⼀国家的历史结束。\n南明时期\n北京沦陷后，史可法等⼈在南京拥⽴福王朱由崧，建⽴弘光政权，即弘光帝，弘光帝死后，鲁王朱\n以海于浙江绍兴监国；⽽唐王朱⾀键在郑芝⻰等⼈的拥⽴下，于福建福州称帝，即隆武帝。⽽两个\n南明政权为争夺正统地位互相攻伐。1651年在⾈⼭群岛沦陷后，鲁王朱以海在张名振、张煌⾔陪同\n下，赴厦⻔依靠郑成功，不久病死在⾦⻔。隆武帝屡议出师北伐，因得不到郑芝⻰的⽀持⽽⽆疾⽽\n终。1646年，清军攻占浙江与福建，鲁王逃亡海外，隆武帝于汀州逃

## Step 6. Chat

我们还可以加入 `Memory` 模块并替换使用 `ConversationalRetrievalChain` 来实现记忆化的对话式查询。

In [13]:
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory, combine_docs_chain_kwargs={"prompt": QA_CHAIN_PROMPT})
qa.invoke(QUESTION1)

[INFO] [03-28 18:25:43] openapi_requestor.py:336 [t:8094817088]: requesting llm api endpoint: /chat/completions_pro
[INFO] [03-28 18:25:44] openapi_requestor.py:336 [t:8094817088]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-28 18:25:45] openapi_requestor.py:336 [t:8094817088]: requesting llm api endpoint: /chat/completions_pro
[INFO] [03-28 18:25:48] openapi_requestor.py:336 [t:8094817088]: requesting llm api endpoint: /chat/completions_pro


{'question': '明朝的开国皇帝是谁',
 'chat_history': [SystemMessage(content='')],
 'answer': '明朝的开国皇帝是朱元璋。感谢你的提问。'}

In [ ]:
qa.invoke(QUESTION2)